In [1]:
class ValidationError(Exception):
    """Custom exception for validation errors."""
    pass

def validate(data, template, prefix=""):
    """
    Validate a dictionary against a template.
    
    Args:
        data (dict): The data to validate.
        template (dict): The template to validate against.
        prefix (str): Used internally to track the current key path.
    
    Returns:
        bool: True if validation passes.
    
    Raises:
        ValidationError: If validation fails with a specific error message.
    """
    for key, value_type in template.items():
        current_path = f"{prefix}.{key}" if prefix else key
        
        # Check if the key exists in the data
        if key not in data:
            raise ValidationError(f"mismatched keys: {current_path}")
        
        # Check the type of the value
        if isinstance(value_type, dict):  # Nested dictionary
            if not isinstance(data[key], dict):
                raise ValidationError(f"bad type: {current_path}")
            validate(data[key], value_type, current_path)  # Recurse for nested dict
        else:
            if not isinstance(data[key], value_type):
                raise ValidationError(f"bad type: {current_path}")
    
    return True  # If no errors, validation is successful


In [2]:
template = {
    'user_id': int,
    'name': {
        'first': str,
        'last': str,
    },
    'bio': {
        'dob': {
            'year': int,
            'month': int,
            'day': int
        },
        'birthplace': {
            'country': str,
            'city': str
        }
    }

}


# Example dictionaries
john = {
    'user_id': 100,
    'name': {
        'first': 'John',
        'last': 'Cleese'
    },
    'bio': {
        'dob': {
            'year': 1939,
            'month': 11,
            'day': 27
        },
        'birthplace': {
            'country': 'United Kingdom',
            'city': 'Weston-super-Mare'
        }
    }
}

eric = {
    'user_id': 101,
    'name': {
        'first': 'Eric',
        'last': 'Idle'
    },
    'bio': {
        'dob': {
            'year': 1943,
            'month': 3,
            'day': 29
        },
        'birthplace': {
            'country': 'United Kingdom'
        }
    }
}

michael = {
    'user_id': 102,
    'name': {
        'first': 'Michael',
        'last': 'Palin'
    },
    'bio': {
        'dob': {
            'year': 1943,
            'month': 'May',
            'day': 5
        },
        'birthplace': {
            'country': 'United Kingdom',
            'city': 'Sheffield'
        }
    }
}

# Validation calls
try:
    print(validate(john, template))  # True
except ValidationError as e:
    print(f"Validation failed: {e}")

try:
    print(validate(eric, template))  # Exception: mismatched keys: bio.birthplace.city
except ValidationError as e:
    print(f"Validation failed: {e}")

try:
    print(validate(michael, template))  # Exception: bad type: bio.dob.month
except ValidationError as e:
    print(f"Validation failed: {e}")


True
Validation failed: mismatched keys: bio.birthplace.city
Validation failed: bad type: bio.dob.month
